In [1]:
%pip install -qU langchain_core langchain_openai chroma sentence-transformers docling langchain-text-splitters "langchain-chroma>=0.1.2" langchain_huggingface https://gradio-builds.s3.amazonaws.com/a0c487cd57a217775f0d1bc77c041b7cd516cc8a/gradio-3.41.2-py3-none-any.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 

# Introduction to Retrieval Augmented Generation (RAG)

__Authored by__: Enrique Noriega-Atala
__Last edited__: 10/07/2024

Retrieval Augmented Generation is a method to prompt the LLM to elicit responses based on information retrieved from a data base, hence _retrieval_.

RAG is a useful method to reduce LLM hallucinations and to _interface_ with documents in a more natural way, akin to a conversation rather than a search engine.

## Step 1: Document Parsing

In [2]:
from docling.document_converter import DocumentConverter

source = "https://arxiv.org/pdf/1706.03762"  # PDF path or URL
converter = DocumentConverter()
result = converter.convert_single(source)
markdown = result.render_as_markdown()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/3.49k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

...

otslp_all_standard_094_clean.check:   0%|          | 0.00/213M [00:00<?, ?B/s]

(…)del_artifacts/tableformer/tm_config.json:   0%|          | 0.00/7.09k [00:00<?, ?B/s]

otslp_all_fast.check:   0%|          | 0.00/146M [00:00<?, ?B/s]

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [3]:
from IPython.display import display, Markdown, Latex

display(Markdown(markdown))

## Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works.

## Attention Is All You Need

Ashish Vaswani ∗ Google Brain avaswani@google.com

Noam Shazeer ∗ Google Brain noam@google.com

Niki Parmar ∗ Google Research nikip@google.com

Jakob Uszkoreit ∗ Google Research usz@google.com

Llion Jones ∗ Google Research llion@google.com

Aidan N. Gomez ∗ † University of Toronto aidan@cs.toronto.edu

Łukasz Kaiser ∗ Google Brain lukaszkaiser@google.com

Illia Polosukhin ∗ ‡ illia.polosukhin@gmail.com

## Abstract

The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 Englishto-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data.

## 1 Introduction

Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks in particular, have been firmly established as state of the art approaches in sequence modeling and transduction problems such as language modeling and machine translation [35, 2, 5]. Numerous efforts have since continued to push the boundaries of recurrent language models and encoder-decoder architectures [38, 24, 15].

Recurrent models typically factor computation along the symbol positions of the input and output sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden states h$_{t}$ , as a function of the previous hidden state h$_{t}$$_{-}$$_{1}$ and the input for position t . This inherently sequential nature precludes parallelization within training examples, which becomes critical at longer sequence lengths, as memory constraints limit batching across examples. Recent work has achieved significant improvements in computational efficiency through factorization tricks [21] and conditional computation [32], while also improving model performance in case of the latter. The fundamental constraint of sequential computation, however, remains.

Attention mechanisms have become an integral part of compelling sequence modeling and transduction models in various tasks, allowing modeling of dependencies without regard to their distance in the input or output sequences [2, 19]. In all but a few cases [27], however, such attention mechanisms are used in conjunction with a recurrent network.

In this work we propose the Transformer, a model architecture eschewing recurrence and instead relying entirely on an attention mechanism to draw global dependencies between input and output. The Transformer allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs.

## 2 Background

The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU [16], ByteNet [18] and ConvS2S [9], all of which use convolutional neural networks as basic building block, computing hidden representations in parallel for all input and output positions. In these models, the number of operations required to relate signals from two arbitrary input or output positions grows in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes it more difficult to learn dependencies between distant positions [12]. In the Transformer this is reduced to a constant number of operations, albeit at the cost of reduced effective resolution due to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as described in section 3.2.

Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence. Self-attention has been used successfully in a variety of tasks including reading comprehension, abstractive summarization, textual entailment and learning task-independent sentence representations [4, 27, 28, 22].

End-to-end memory networks are based on a recurrent attention mechanism instead of sequencealigned recurrence and have been shown to perform well on simple-language question answering and language modeling tasks [34].

To the best of our knowledge, however, the Transformer is the first transduction model relying entirely on self-attention to compute representations of its input and output without using sequencealigned RNNs or convolution. In the following sections, we will describe the Transformer, motivate self-attention and discuss its advantages over models such as [17, 18] and [9].

## 3 Model Architecture

Most competitive neural sequence transduction models have an encoder-decoder structure [5, 2, 35]. Here, the encoder maps an input sequence of symbol representations ( x$_{1}$, ..., x$_{n}$ ) to a sequence of continuous representations z = ( z$_{1}$, ..., z$_{n}$ ) . Given z , the decoder then generates an output sequence ( y$_{1}$, ..., y$_{m}$ ) of symbols one element at a time. At each step the model is auto-regressive [10], consuming the previously generated symbols as additional input when generating the next.

Figure 1: The Transformer - model architecture.
<!-- image -->

The Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1, respectively.

## 3.1 Encoder and Decoder Stacks

Encoder: The encoder is composed of a stack of N = 6 identical layers. Each layer has two sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, positionwise fully connected feed-forward network. We employ a residual connection [11] around each of the two sub-layers, followed by layer normalization [1]. That is, the output of each sub-layer is LayerNorm( x + Sublayer( x )) , where Sublayer( x ) is the function implemented by the sub-layer itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding layers, produce outputs of dimension d$_{model}$ = 512 .

Decoder: The decoder is also composed of a stack of N = 6 identical layers. In addition to the two sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head attention over the output of the encoder stack. Similar to the encoder, we employ residual connections around each of the sub-layers, followed by layer normalization. We also modify the self-attention sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This masking, combined with fact that the output embeddings are offset by one position, ensures that the predictions for position i can depend only on the known outputs at positions less than i .

## 3.2 Attention

An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum

Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several attention layers running in parallel.

Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several attention layers running in parallel.
<!-- image -->

of the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key.

## 3.2.1 Scaled Dot-Product Attention

We call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of queries and keys of dimension d$_{k}$ , and values of dimension d$_{v}$ . We compute the dot products of the query with all keys, divide each by √ d$_{k}$ , and apply a softmax function to obtain the weights on the values.

In practice, we compute the attention function on a set of queries simultaneously, packed together into a matrix Q . The keys and values are also packed together into matrices K and V . We compute the matrix of outputs as:

The two most commonly used attention functions are additive attention [2], and dot-product (multiplicative) attention. Dot-product attention is identical to our algorithm, except for the scaling factor of 1 √ $_{d$_{k}$}$. Additive attention computes the compatibility function using a feed-forward network with a single hidden layer. While the two are similar in theoretical complexity, dot-product attention is much faster and more space-efficient in practice, since it can be implemented using highly optimized matrix multiplication code.

While for small values of d$_{k}$ the two mechanisms perform similarly, additive attention outperforms dot product attention without scaling for larger values of d$_{k}$ [3]. We suspect that for large values of d$_{k}$ , the dot products grow large in magnitude, pushing the softmax function into regions where it has extremely small gradients $^{4}$. To counteract this effect, we scale the dot products by 1 √ $_{d$_{k}$}$.

## 3.2.2 Multi-Head Attention

Instead of performing a single attention function with d$_{model}$ -dimensional keys, values and queries, we found it beneficial to linearly project the queries, keys and values h times with different, learned linear projections to d$_{k}$ , d$_{k}$ and d$_{v}$ dimensions, respectively. On each of these projected versions of queries, keys and values we then perform the attention function in parallel, yielding d$_{v}$ -dimensional

output values. These are concatenated and once again projected, resulting in the final values, as depicted in Figure 2.

Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averaging inhibits this.

MultiHead( Q,K,V ) = Concat(head$_{1}$ ,..., head$_{h}$) W O where head$_{i}$ = Attention( QW Q i ,KW K i ,V W V i )

Where the projections are parameter matrices W Q i ∈$_{R}$ d$_{model}$ × $^{d$_{k}$}$, W K i ∈$_{R}$ d$_{model}$ × $^{d$_{k}$}$, W V i ∈$_{R}$ d$_{model}$ × d$_{v}$ and W O ∈$_{R}$ hd$_{v}$ × $^{d$_{model}$}$.

In this work we employ h = 8 parallel attention layers, or heads. For each of these we use d$_{k}$ = d$_{v}$ = d$_{model}$/h = 64 . Due to the reduced dimension of each head, the total computational cost is similar to that of single-head attention with full dimensionality.

## 3.2.3 Applications of Attention in our Model

The Transformer uses multi-head attention in three different ways:

· In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9].

· The encoder contains self-attention layers. In a self-attention layer all of the keys, values and queries come from the same place, in this case, the output of the previous layer in the encoder. Each position in the encoder can attend to all positions in the previous layer of the encoder.

· Similarly, self-attention layers in the decoder allow each position in the decoder to attend to all positions in the decoder up to and including that position. We need to prevent leftward information flow in the decoder to preserve the auto-regressive property. We implement this inside of scaled dot-product attention by masking out (setting to -∞ ) all values in the input of the softmax which correspond to illegal connections. See Figure 2.

## 3.3 Position-wise Feed-Forward Networks

In addition to attention sub-layers, each of the layers in our encoder and decoder contains a fully connected feed-forward network, which is applied to each position separately and identically. This consists of two linear transformations with a ReLU activation in between.

While the linear transformations are the same across different positions, they use different parameters from layer to layer. Another way of describing this is as two convolutions with kernel size 1. The dimensionality of input and output is d$_{model}$ = 512 , and the inner-layer has dimensionality d$_{ff}$ = 2048 .

## 3.4 Embeddings and Softmax

Similarly to other sequence transduction models, we use learned embeddings to convert the input tokens and output tokens to vectors of dimension d$_{model}$ . We also use the usual learned linear transformation and softmax function to convert the decoder output to predicted next-token probabilities. In our model, we share the same weight matrix between the two embedding layers and the pre-softmax linear transformation, similar to [30]. In the embedding layers, we multiply those weights by √ d$_{model}$ .

Table 1: Maximum path lengths, per-layer complexity and minimum number of sequential operations for different layer types. n is the sequence length, d is the representation dimension, k is the kernel size of convolutions and r the size of the neighborhood in restricted self-attention.
| Layer Type                  | Complexity per Layer   | Sequential Operations   | Maximum Path Length   |
|-----------------------------|------------------------|-------------------------|-----------------------|
| Self-Attention              | O ( n 2 · d )          | O (1)                   | O (1)                 |
| Recurrent                   | O ( n · d $^{2}$)      | O ( n )                 | O ( n )               |
| Convolutional               | O ( k · n · d $^{2}$)  | O (1)                   | O ( log$_{k}$ ( n ))  |
| Self-Attention (restricted) | O ( r · n · d )        | O (1)                   | O ( n/r )             |

## 3.5 Positional Encoding

Since our model contains no recurrence and no convolution, in order for the model to make use of the order of the sequence, we must inject some information about the relative or absolute position of the tokens in the sequence. To this end, we add "positional encodings" to the input embeddings at the bottoms of the encoder and decoder stacks. The positional encodings have the same dimension d$_{model}$ as the embeddings, so that the two can be summed. There are many choices of positional encodings, learned and fixed [9].

In this work, we use sine and cosine functions of different frequencies:

where pos is the position and i is the dimension. That is, each dimension of the positional encoding corresponds to a sinusoid. The wavelengths form a geometric progression from 2 π to 10000 · 2 π . We chose this function because we hypothesized it would allow the model to easily learn to attend by relative positions, since for any fixed offset k , PE$_{pos}$$_{+}$$_{k}$ can be represented as a linear function of PE$_{pos}$ .

We also experimented with using learned positional embeddings [9] instead, and found that the two versions produced nearly identical results (see Table 3 row (E)). We chose the sinusoidal version because it may allow the model to extrapolate to sequence lengths longer than the ones encountered during training.

## 4 Why Self-Attention

In this section we compare various aspects of self-attention layers to the recurrent and convolutional layers commonly used for mapping one variable-length sequence of symbol representations ( x$_{1}$, ..., x$_{n}$ ) to another sequence of equal length ( z$_{1}$, ..., z$_{n}$ ) , with x$_{i}$, z$_{i}$ ∈ R $^{d}$, such as a hidden layer in a typical sequence transduction encoder or decoder. Motivating our use of self-attention we consider three desiderata.

One is the total computational complexity per layer. Another is the amount of computation that can be parallelized, as measured by the minimum number of sequential operations required.

The third is the path length between long-range dependencies in the network. Learning long-range dependencies is a key challenge in many sequence transduction tasks. One key factor affecting the ability to learn such dependencies is the length of the paths forward and backward signals have to traverse in the network. The shorter these paths between any combination of positions in the input and output sequences, the easier it is to learn long-range dependencies [12]. Hence we also compare the maximum path length between any two input and output positions in networks composed of the different layer types.

As noted in Table 1, a self-attention layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires O ( n ) sequential operations. In terms of computational complexity, self-attention layers are faster than recurrent layers when the sequence

length n is smaller than the representation dimensionality d , which is most often the case with sentence representations used by state-of-the-art models in machine translations, such as word-piece [38] and byte-pair [31] representations. To improve computational performance for tasks involving very long sequences, self-attention could be restricted to considering only a neighborhood of size r in the input sequence centered around the respective output position. This would increase the maximum path length to O ( n/r ) . We plan to investigate this approach further in future work.

A single convolutional layer with kernel width k < n does not connect all pairs of input and output positions. Doing so requires a stack of O ( n/k ) convolutional layers in the case of contiguous kernels, or O ( log$_{k}$ ( n )) in the case of dilated convolutions [18], increasing the length of the longest paths between any two positions in the network. Convolutional layers are generally more expensive than recurrent layers, by a factor of k . Separable convolutions [6], however, decrease the complexity considerably, to O ( k · n · d + n · d $^{2}$) . Even with k = n , however, the complexity of a separable convolution is equal to the combination of a self-attention layer and a point-wise feed-forward layer, the approach we take in our model.

As side benefit, self-attention could yield more interpretable models. We inspect attention distributions from our models and present and discuss examples in the appendix. Not only do individual attention heads clearly learn to perform different tasks, many appear to exhibit behavior related to the syntactic and semantic structure of the sentences.

## 5 Training

This section describes the training regime for our models.

## 5.1 Training Data and Batching

We trained on the standard WMT 2014 English-German dataset consisting of about 4.5 million sentence pairs. Sentences were encoded using byte-pair encoding [3], which has a shared sourcetarget vocabulary of about 37000 tokens. For English-French, we used the significantly larger WMT 2014 English-French dataset consisting of 36M sentences and split tokens into a 32000 word-piece vocabulary [38]. Sentence pairs were batched together by approximate sequence length. Each training batch contained a set of sentence pairs containing approximately 25000 source tokens and 25000 target tokens.

## 5.2 Hardware and Schedule

We trained our models on one machine with 8 NVIDIA P100 GPUs. For our base models using the hyperparameters described throughout the paper, each training step took about 0.4 seconds. We trained the base models for a total of 100,000 steps or 12 hours. For our big models,(described on the bottom line of table 3), step time was 1.0 seconds. The big models were trained for 300,000 steps (3.5 days).

## 5.3 Optimizer

We used the Adam optimizer [20] with β$_{1}$ = 0 . 9 , β$_{2}$ = 0 . 98 and ϵ = 10 - $^{9}$. We varied the learning rate over the course of training, according to the formula:

This corresponds to increasing the learning rate linearly for the first warmup _ steps training steps, and decreasing it thereafter proportionally to the inverse square root of the step number. We used warmup _ steps = 4000 .

## 5.4 Regularization

We employ three types of regularization during training:

Table 2: The Transformer achieves better BLEU scores than previous state-of-the-art models on the English-to-German and English-to-French newstest2014 tests at a fraction of the training cost.
| Model                           | BLEU   | BLEU   | Training Cost (FLOPs)   | Training Cost (FLOPs)   |
|---------------------------------|--------|--------|-------------------------|-------------------------|
|                                 | EN-DE  | EN-FR  | EN-DE                   | EN-FR                   |
| ByteNet [18]                    | 23.75  |        |                         |                         |
| Deep-Att + PosUnk [39]          |        | 39.2   |                         | 1 . 0 · 10 20           |
| GNMT + RL [38]                  | 24.6   | 39.92  | 2 . 3 · 10 19           | 1 . 4 · 10 20           |
| ConvS2S [9]                     | 25.16  | 40.46  | 9 . 6 · 10 18           | 1 . 5 · 10 20           |
| MoE [32]                        | 26.03  | 40.56  | 2 . 0 · 10 19           | 1 . 2 · 10 20           |
| Deep-Att + PosUnk Ensemble [39] |        | 40.4   |                         | 8 . 0 · 10 20           |
| GNMT + RL Ensemble [38]         | 26.30  | 41.16  | 1 . 8 · 10 20           | 1 . 1 · 10 21           |
| ConvS2S Ensemble [9]            | 26.36  | 41.29  | 7 . 7 · 10 19           | 1 . 2 · 10 21           |
| Transformer (base model)        | 27.3   | 38.1   | 3 . 3 · 10              | 18                      |
| Transformer (big)               | 28.4   | 41.8   | 2 . 3 · 10 19           | 2 . 3 · 10 19           |

Residual Dropout We apply dropout [33] to the output of each sub-layer, before it is added to the sub-layer input and normalized. In addition, we apply dropout to the sums of the embeddings and the positional encodings in both the encoder and decoder stacks. For the base model, we use a rate of P$_{drop}$ = 0 . 1 .

Label Smoothing During training, we employed label smoothing of value ϵ$_{ls}$ = 0 . 1 [36]. This hurts perplexity, as the model learns to be more unsure, but improves accuracy and BLEU score.

## 6 Results

## 6.1 Machine Translation

On the WMT 2014 English-to-German translation task, the big transformer model (Transformer (big) in Table 2) outperforms the best previously reported models (including ensembles) by more than 2 . 0 BLEU, establishing a new state-of-the-art BLEU score of 28 . 4 . The configuration of this model is listed in the bottom line of Table 3. Training took 3 . 5 days on 8 P100 GPUs. Even our base model surpasses all previously published models and ensembles, at a fraction of the training cost of any of the competitive models.

On the WMT 2014 English-to-French translation task, our big model achieves a BLEU score of 41 . 0 , outperforming all of the previously published single models, at less than 1 / 4 the training cost of the previous state-of-the-art model. The Transformer (big) model trained for English-to-French used dropout rate P$_{drop}$ = 0 . 1 , instead of 0 . 3 .

For the base models, we used a single model obtained by averaging the last 5 checkpoints, which were written at 10-minute intervals. For the big models, we averaged the last 20 checkpoints. We used beam search with a beam size of 4 and length penalty α = 0 . 6 [38]. These hyperparameters were chosen after experimentation on the development set. We set the maximum output length during inference to input length + 50 , but terminate early when possible [38].

Table 2 summarizes our results and compares our translation quality and training costs to other model architectures from the literature. We estimate the number of floating point operations used to train a model by multiplying the training time, the number of GPUs used, and an estimate of the sustained single-precision floating-point capacity of each GPU $^{5}$.

## 6.2 Model Variations

To evaluate the importance of different components of the Transformer, we varied our base model in different ways, measuring the change in performance on English-to-German translation on the

Table 3: Variations on the Transformer architecture. Unlisted values are identical to those of the base model. All metrics are on the English-to-German translation development set, newstest2013. Listed perplexities are per-wordpiece, according to our byte-pair encoding, and should not be compared to per-word perplexities.

Table 3: Variations on the Transformer architecture. Unlisted values are identical to those of the base model. All metrics are on the English-to-German translation development set, newstest2013. Listed perplexities are per-wordpiece, according to our byte-pair encoding, and should not be compared to per-word perplexities.
|      | N                                             | d$_{model}$                                   | d$_{ff}$                                      | h                                             | d$_{k}$                                       | d$_{v}$                                       | P$_{drop}$                                    | ϵ$_{ls}$                                      | train steps                                   |   PPL (dev) |   BLEU (dev) | params × 10 6   |
|------|-----------------------------------------------|-----------------------------------------------|-----------------------------------------------|-----------------------------------------------|-----------------------------------------------|-----------------------------------------------|-----------------------------------------------|-----------------------------------------------|-----------------------------------------------|-------------|--------------|-----------------|
| base | 6                                             | 512                                           | 2048                                          | 8                                             | 64                                            | 64                                            | 0.1                                           | 0.1                                           | 100K                                          |        4.92 |         25.8 | 65              |
|      |                                               |                                               |                                               | 1                                             | 512                                           | 512                                           |                                               |                                               |                                               |        5.29 |         24.9 |                 |
|      |                                               |                                               |                                               | 4                                             | 128                                           | 128                                           |                                               |                                               |                                               |        5    |         25.5 |                 |
| (A)  |                                               |                                               |                                               | 16                                            | 32                                            | 32                                            |                                               |                                               |                                               |        4.91 |         25.8 |                 |
| (A)  |                                               |                                               | 32                                            | 16                                            | 16                                            |                                               |                                               |                                               |                                               |        5.01 |         25.4 |                 |
|      |                                               |                                               |                                               |                                               | 16                                            |                                               |                                               |                                               |                                               |        5.16 |         25.1 | 58              |
| (B)  |                                               |                                               |                                               |                                               | 32                                            |                                               |                                               |                                               |                                               |        5.01 |         25.4 | 60              |
| (C)  | 2                                             |                                               |                                               |                                               |                                               |                                               |                                               |                                               |                                               |        6.11 |         23.7 | 36              |
| (C)  | 4                                             |                                               |                                               |                                               |                                               |                                               |                                               |                                               |                                               |        5.19 |         25.3 | 50              |
| (C)  | 8                                             |                                               |                                               |                                               |                                               |                                               |                                               |                                               |                                               |        4.88 |         25.5 | 80              |
|      |                                               | 256                                           |                                               |                                               | 32                                            | 32                                            |                                               |                                               |                                               |        5.75 |         24.5 | 28              |
|      |                                               | 1024                                          |                                               | 128                                           |                                               | 128                                           |                                               |                                               |                                               |        4.66 |         26   | 168             |
|      |                                               |                                               | 1024                                          |                                               |                                               |                                               |                                               |                                               |                                               |        5.12 |         25.4 | 53              |
|      | 4096                                          | 4096                                          |                                               |                                               |                                               |                                               |                                               |                                               |                                               |        4.75 |         26.2 | 90              |
|      |                                               |                                               |                                               |                                               |                                               |                                               | 0.0                                           |                                               |                                               |        5.77 |         24.6 |                 |
| (D)  |                                               |                                               |                                               |                                               |                                               |                                               | 0.2                                           |                                               |                                               |        4.95 |         25.5 |                 |
|      |                                               |                                               |                                               |                                               |                                               |                                               |                                               | 0.0                                           |                                               |        4.67 |         25.3 |                 |
| (E)  | 0.2 positional embedding instead of sinusoids | 0.2 positional embedding instead of sinusoids | 0.2 positional embedding instead of sinusoids | 0.2 positional embedding instead of sinusoids | 0.2 positional embedding instead of sinusoids | 0.2 positional embedding instead of sinusoids | 0.2 positional embedding instead of sinusoids | 0.2 positional embedding instead of sinusoids | 0.2 positional embedding instead of sinusoids |        4.92 |         25.7 |                 |
| big  | 6                                             | 1024                                          | 4096                                          | 16                                            |                                               |                                               | 0.3                                           |                                               | 300K                                          |        4.33 |         26.4 | 213             |

development set, newstest2013. We used beam search as described in the previous section, but no checkpoint averaging. We present these results in Table 3.

In Table 3 rows (A), we vary the number of attention heads and the attention key and value dimensions, keeping the amount of computation constant, as described in Section 3.2.2. While single-head attention is 0.9 BLEU worse than the best setting, quality also drops off with too many heads.

In Table 3 rows (B), we observe that reducing the attention key size d$_{k}$ hurts model quality. This suggests that determining compatibility is not easy and that a more sophisticated compatibility function than dot product may be beneficial. We further observe in rows (C) and (D) that, as expected, bigger models are better, and dropout is very helpful in avoiding over-fitting. In row (E) we replace our sinusoidal positional encoding with learned positional embeddings [9], and observe nearly identical results to the base model.

## 6.3 English Constituency Parsing

To evaluate if the Transformer can generalize to other tasks we performed experiments on English constituency parsing. This task presents specific challenges: the output is subject to strong structural constraints and is significantly longer than the input. Furthermore, RNN sequence-to-sequence models have not been able to attain state-of-the-art results in small-data regimes [37].

We trained a 4-layer transformer with d$_{model}$ = 1024 on the Wall Street Journal (WSJ) portion of the Penn Treebank [25], about 40K training sentences. We also trained it in a semi-supervised setting, using the larger high-confidence and BerkleyParser corpora from with approximately 17M sentences [37]. We used a vocabulary of 16K tokens for the WSJ only setting and a vocabulary of 32K tokens for the semi-supervised setting.

We performed only a small number of experiments to select the dropout, both attention and residual (section 5.4), learning rates and beam size on the Section 22 development set, all other parameters remained unchanged from the English-to-German base translation model. During inference, we

Table 4: The Transformer generalizes well to English constituency parsing (Results are on Section 23 of WSJ)
| Parser                              | Training                 |   WSJ 23 F1 |
|-------------------------------------|--------------------------|-------------|
| Vinyals & Kaiser el al. (2014) [37] | WSJ only, discriminative |        88.3 |
| Petrov et al. (2006) [29]           | WSJ only, discriminative |        90.4 |
| Zhu et al. (2013) [40]              | WSJ only, discriminative |        90.4 |
| Dyer et al. (2016) [8]              | WSJ only, discriminative |        91.7 |
| Transformer (4 layers)              | WSJ only, discriminative |        91.3 |
| Zhu et al. (2013) [40]              | semi-supervised          |        91.3 |
| Huang & Harper (2009) [14]          | semi-supervised          |        91.3 |
| McClosky et al. (2006) [26]         | semi-supervised          |        92.1 |
| Vinyals & Kaiser el al. (2014) [37] | semi-supervised          |        92.1 |
| Transformer (4 layers)              | semi-supervised          |        92.7 |
| Luong et al. (2015) [23]            | multi-task               |        93   |
| Dyer et al. (2016) [8]              | generative               |        93.3 |

increased the maximum output length to input length + 300 . We used a beam size of 21 and α = 0 . 3 for both WSJ only and the semi-supervised setting.

Our results in Table 4 show that despite the lack of task-specific tuning our model performs surprisingly well, yielding better results than all previously reported models with the exception of the Recurrent Neural Network Grammar [8].

In contrast to RNN sequence-to-sequence models [37], the Transformer outperforms the BerkeleyParser [29] even when training only on the WSJ training set of 40K sentences.

## 7 Conclusion

In this work, we presented the Transformer, the first sequence transduction model based entirely on attention, replacing the recurrent layers most commonly used in encoder-decoder architectures with multi-headed self-attention.

For translation tasks, the Transformer can be trained significantly faster than architectures based on recurrent or convolutional layers. On both WMT 2014 English-to-German and WMT 2014 English-to-French translation tasks, we achieve a new state of the art. In the former task our best model outperforms even all previously reported ensembles.

We are excited about the future of attention-based models and plan to apply them to other tasks. We plan to extend the Transformer to problems involving input and output modalities other than text and to investigate local, restricted attention mechanisms to efficiently handle large inputs and outputs such as images, audio and video. Making generation less sequential is another research goals of ours.

The code we used to train and evaluate our models is available at https://github.com/ tensorflow/tensor2tensor .

Acknowledgements We are grateful to Nal Kalchbrenner and Stephan Gouws for their fruitful comments, corrections and inspiration.

## References

[1] Jimmy Lei Ba, Jamie Ryan Kiros, and Geoffrey E Hinton. Layer normalization. arXiv preprint arXiv:1607.06450 , 2016.

[2] Dzmitry Bahdanau, Kyunghyun Cho, and Yoshua Bengio. Neural machine translation by jointly learning to align and translate. CoRR , abs/1409.0473, 2014.

[3] Denny Britz, Anna Goldie, Minh-Thang Luong, and Quoc V. Le. Massive exploration of neural machine translation architectures. CoRR , abs/1703.03906, 2017.

[4] Jianpeng Cheng, Li Dong, and Mirella Lapata. Long short-term memory-networks for machine reading. arXiv preprint arXiv:1601.06733 , 2016.

[5] Kyunghyun Cho, Bart van Merrienboer, Caglar Gulcehre, Fethi Bougares, Holger Schwenk, and Yoshua Bengio. Learning phrase representations using rnn encoder-decoder for statistical machine translation. CoRR , abs/1406.1078, 2014.

[6] Francois Chollet. Xception: Deep learning with depthwise separable convolutions. arXiv preprint arXiv:1610.02357 , 2016.

[7] Junyoung Chung, Çaglar Gülçehre, Kyunghyun Cho, and Yoshua Bengio. Empirical evaluation of gated recurrent neural networks on sequence modeling. CoRR , abs/1412.3555, 2014.

[8] Chris Dyer, Adhiguna Kuncoro, Miguel Ballesteros, and Noah A. Smith. Recurrent neural network grammars. In Proc. of NAACL , 2016.

[9] Jonas Gehring, Michael Auli, David Grangier, Denis Yarats, and Yann N. Dauphin. Convolutional sequence to sequence learning. arXiv preprint arXiv:1705.03122v2 , 2017.

[10] Alex Graves. Generating sequences with recurrent neural networks. arXiv preprint arXiv:1308.0850 , 2013.

[11] Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun. Deep residual learning for image recognition. In Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition , pages 770-778, 2016.

[12] Sepp Hochreiter, Yoshua Bengio, Paolo Frasconi, and Jürgen Schmidhuber. Gradient flow in recurrent nets: the difficulty of learning long-term dependencies, 2001.

[13] Sepp Hochreiter and Jürgen Schmidhuber. Long short-term memory. Neural computation , 9(8):1735-1780, 1997.

[14] Zhongqiang Huang and Mary Harper. Self-training PCFG grammars with latent annotations across languages. In Proceedings of the 2009 Conference on Empirical Methods in Natural Language Processing , pages 832-841. ACL, August 2009.

[15] Rafal Jozefowicz, Oriol Vinyals, Mike Schuster, Noam Shazeer, and Yonghui Wu. Exploring the limits of language modeling. arXiv preprint arXiv:1602.02410 , 2016.

[16] Łukasz Kaiser and Samy Bengio. Can active memory replace attention? In Advances in Neural Information Processing Systems, (NIPS) , 2016.

[17] Łukasz Kaiser and Ilya Sutskever. Neural GPUs learn algorithms. In International Conference on Learning Representations (ICLR) , 2016.

[18] Nal Kalchbrenner, Lasse Espeholt, Karen Simonyan, Aaron van den Oord, Alex Graves, and Koray Kavukcuoglu. Neural machine translation in linear time. arXiv preprint arXiv:1610.10099v2 , 2017.

[19] Yoon Kim, Carl Denton, Luong Hoang, and Alexander M. Rush. Structured attention networks. In International Conference on Learning Representations , 2017.

[20] Diederik Kingma and Jimmy Ba. Adam: A method for stochastic optimization. In ICLR , 2015.

[21] Oleksii Kuchaiev and Boris Ginsburg. Factorization tricks for LSTM networks. arXiv preprint arXiv:1703.10722 , 2017.

[22] Zhouhan Lin, Minwei Feng, Cicero Nogueira dos Santos, Mo Yu, Bing Xiang, Bowen Zhou, and Yoshua Bengio. A structured self-attentive sentence embedding. arXiv preprint arXiv:1703.03130 , 2017.

[23] Minh-Thang Luong, Quoc V. Le, Ilya Sutskever, Oriol Vinyals, and Lukasz Kaiser. Multi-task sequence to sequence learning. arXiv preprint arXiv:1511.06114 , 2015.

[24] Minh-Thang Luong, Hieu Pham, and Christopher D Manning. Effective approaches to attentionbased neural machine translation. arXiv preprint arXiv:1508.04025 , 2015.

[25] Mitchell P Marcus, Mary Ann Marcinkiewicz, and Beatrice Santorini. Building a large annotated corpus of english: The penn treebank. Computational linguistics , 19(2):313-330, 1993.

[26] David McClosky, Eugene Charniak, and Mark Johnson. Effective self-training for parsing. In Proceedings of the Human Language Technology Conference of the NAACL, Main Conference , pages 152-159. ACL, June 2006.

[27] Ankur Parikh, Oscar Täckström, Dipanjan Das, and Jakob Uszkoreit. A decomposable attention model. In Empirical Methods in Natural Language Processing , 2016.

[28] Romain Paulus, Caiming Xiong, and Richard Socher. A deep reinforced model for abstractive summarization. arXiv preprint arXiv:1705.04304 , 2017.

[29] Slav Petrov, Leon Barrett, Romain Thibaux, and Dan Klein. Learning accurate, compact, and interpretable tree annotation. In Proceedings of the 21st International Conference on Computational Linguistics and 44th Annual Meeting of the ACL , pages 433-440. ACL, July 2006.

[30] Ofir Press and Lior Wolf. Using the output embedding to improve language models. arXiv preprint arXiv:1608.05859 , 2016.

[31] Rico Sennrich, Barry Haddow, and Alexandra Birch. Neural machine translation of rare words with subword units. arXiv preprint arXiv:1508.07909 , 2015.

[32] Noam Shazeer, Azalia Mirhoseini, Krzysztof Maziarz, Andy Davis, Quoc Le, Geoffrey Hinton, and Jeff Dean. Outrageously large neural networks: The sparsely-gated mixture-of-experts layer. arXiv preprint arXiv:1701.06538 , 2017.

[33] Nitish Srivastava, Geoffrey E Hinton, Alex Krizhevsky, Ilya Sutskever, and Ruslan Salakhutdinov. Dropout: a simple way to prevent neural networks from overfitting. Journal of Machine Learning Research , 15(1):1929-1958, 2014.

[34] Sainbayar Sukhbaatar, Arthur Szlam, Jason Weston, and Rob Fergus. End-to-end memory networks. In C. Cortes, N. D. Lawrence, D. D. Lee, M. Sugiyama, and R. Garnett, editors, Advances in Neural Information Processing Systems 28 , pages 2440-2448. Curran Associates, Inc., 2015.

[35] Ilya Sutskever, Oriol Vinyals, and Quoc VV Le. Sequence to sequence learning with neural networks. In Advances in Neural Information Processing Systems , pages 3104-3112, 2014.

[36] Christian Szegedy, Vincent Vanhoucke, Sergey Ioffe, Jonathon Shlens, and Zbigniew Wojna. Rethinking the inception architecture for computer vision. CoRR , abs/1512.00567, 2015.

[37] Vinyals & Kaiser, Koo, Petrov, Sutskever, and Hinton. Grammar as a foreign language. In Advances in Neural Information Processing Systems , 2015.

[38] Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc V Le, Mohammad Norouzi, Wolfgang Macherey, Maxim Krikun, Yuan Cao, Qin Gao, Klaus Macherey, et al. Google's neural machine translation system: Bridging the gap between human and machine translation. arXiv preprint arXiv:1609.08144 , 2016.

[39] Jie Zhou, Ying Cao, Xuguang Wang, Peng Li, and Wei Xu. Deep recurrent models with fast-forward connections for neural machine translation. CoRR , abs/1606.04199, 2016.

[40] Muhua Zhu, Yue Zhang, Wenliang Chen, Min Zhang, and Jingbo Zhu. Fast and accurate shift-reduce constituent parsing. In Proceedings of the 51st Annual Meeting of the ACL (Volume 1: Long Papers) , pages 434-443. ACL, August 2013.

## Attention Visualizations Input-Input Layer5

Figure 3: An example of the attention mechanism following long-distance dependencies in the encoder self-attention in layer 5 of 6. Many of the attention heads attend to a distant dependency of the verb 'making', completing the phrase 'making...more difficult'. Attentions here shown only for the word 'making'. Different colors represent different heads. Best viewed in color.
<!-- image -->

Input-Input Layer5

Figure 4: Two attention heads, also in layer 5 of 6, apparently involved in anaphora resolution. Top: Full attentions for head 5. Bottom: Isolated attentions from just the word 'its' for attention heads 5 and 6. Note that the attentions are very sharp for this word.
<!-- image -->

Figure 5: Many of the attention heads exhibit behaviour that seems related to the structure of the sentence. We give two such examples above, from two different heads from the encoder self-attention at layer 5 of 6. The heads clearly learned to perform different tasks.
<!-- image -->

## Step 2: Split the document

In [4]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
]

# MD splits
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
md_header_splits = markdown_splitter.split_text(markdown)

# Char-level splits
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size = 250
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(md_header_splits)
splits

[Document(metadata={'Header 2': 'Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works.'}, page_content='## Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works.'),
 Document(metadata={'Header 2': 'Attention Is All You Need'}, page_content='## Attention Is All You Need  \nAshish Vaswani ∗ Google Brain avaswani@google.com  \nNoam Shazeer ∗ Google Brain noam@google.com  \nNiki Parmar ∗ Google Research nikip@google.com  \nJakob Uszkoreit ∗ Google Research usz@google.com'),
 Document(metadata={'Header 2': 'Attention Is All You Need'}, page_content='Llion Jones ∗ Google Research llion@google.com  \nAidan N. Gomez ∗ † University of Toronto aidan@cs.toronto.edu  \nŁukasz Kaiser ∗ Google Brain lukaszkaiser@google.com  \nIllia Polosukhin ∗ ‡ illia.polosukhin@g

In [5]:
print(f'Total number of "chunks": {len(splits)}')

Total number of "chunks": 294


## Step 3: Encode the splits into a _Vector Database_

In order to execute _semantic_ queries, we need to generate _vector representations_ of each split "chunk".

We will do this using `sentence-transformers` and `ChromaDB`

In [6]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

vector_store = Chroma(
    embedding_function=embeddings,
)

# Index the documents using the embeddings model
vector_store.add_documents(splits)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

['464ed4db-f3a5-4776-8bd5-8e07bae46820',
 '2b00e5f3-d2a8-4a42-8515-76370bdd888a',
 'bf1de652-0b87-43d5-a051-97f110cd7200',
 '5a350a4e-493b-429d-8772-9230c1974711',
 'a450f34d-54b7-43af-9fd1-742131920245',
 '9339fcfe-dd28-44a5-885b-f80713793cd8',
 '98065fe6-c735-4b68-bc6e-7167f3af9840',
 '88c375b6-e314-4a35-b70f-0dc06c0be48c',
 'c348da36-434f-4464-9865-a7faf32616e6',
 'ab8a7bb8-d630-472b-8cdc-0a73617188fa',
 '1792878b-ad91-4755-a166-c9b75ad884d1',
 '71810399-0082-4cc6-ad0a-baadffab7fd7',
 '2b357c1c-e601-430c-aa38-61e47fb2aa3e',
 '484e3e06-35da-43ce-90b0-e84b61c2b8bc',
 'ec0b09ed-bf09-470e-af95-bc466f4c165d',
 '04282c57-80c0-4aa0-8bfd-f917df81af01',
 'cf6a266f-27bb-4c96-a25f-c67f749c5c31',
 '1723c24b-9bc4-47ca-97f6-d3600fe7056a',
 'e5c7581e-1e85-4854-a9d1-df17871bb66b',
 '78b68867-2fc2-4605-938d-cde1132aac2c',
 'c31684c1-9249-40a6-926b-a89c69531add',
 'c5e12011-80be-4352-acc3-1cc1b97b16af',
 '7ff8247b-a6e3-417e-aece-566fd6acf276',
 '08733363-0e35-41f7-ba95-4097a895f7a1',
 '22b4dc26-aaca-

In [7]:
# You can do retrieval using sentence similarity
retriever = vector_store.as_retriever()

retriever.invoke("Attention is")

[Document(metadata={'Header 2': '3.2 Attention'}, page_content='## 3.2 Attention  \nAn attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum'),
 Document(metadata={'Header 2': 'Attention Visualizations Input-Input Layer5'}, page_content='that the attentions are very sharp for this word.'),
 Document(metadata={'Header 2': '3.2.2 Multi-Head Attention'}, page_content='## 3.2.2 Multi-Head Attention'),
 Document(metadata={'Header 2': '3.2.1 Scaled Dot-Product Attention'}, page_content='In practice, we compute the attention function on a set of queries simultaneously, packed together into a matrix Q . The keys and values are also packed together into matrices K and V . We compute the matrix of outputs as:')]

In [8]:
# We can configure the parameters of the search, such as the number of chunks returned
retriever = vector_store.as_retriever(search_kwargs={'k':10})
retriever.invoke("Attention is")

[Document(metadata={'Header 2': '3.2 Attention'}, page_content='## 3.2 Attention  \nAn attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum'),
 Document(metadata={'Header 2': 'Attention Visualizations Input-Input Layer5'}, page_content='that the attentions are very sharp for this word.'),
 Document(metadata={'Header 2': '3.2.2 Multi-Head Attention'}, page_content='## 3.2.2 Multi-Head Attention'),
 Document(metadata={'Header 2': '3.2.1 Scaled Dot-Product Attention'}, page_content='In practice, we compute the attention function on a set of queries simultaneously, packed together into a matrix Q . The keys and values are also packed together into matrices K and V . We compute the matrix of outputs as:'),
 Document(metadata={'Header 2': 'References'}, page_content='[27] Ankur Parikh, Oscar Täckström, Dipanjan Das, and Jakob Uszkoreit. A decomposab

## Step 4: Connect the retrieval to an LLM

We will use Verde for this purpose

In [9]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [10]:
from langchain_openai import ChatOpenAI

API_ENDPOINT = "https://llm1.cyverse.ai/v1"

llm = ChatOpenAI(model="Mistral-7B-Instruct-v0.3", base_url=API_ENDPOINT)

llm.invoke("Hello! What can you tell me about yourself?")

AIMessage(content=" Hello! I'm an AI model designed to assist you with a variety of tasks such as answering questions, helping with research, setting reminders, and providing recommendations, among other things. I'm programmed to learn and improve over time based on the interactions I have with users like you. I don't have personal experiences or feelings, but I'm here to make your life easier and more convenient. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 13, 'total_tokens': 105, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/Mistral-7B-Instruct-v0.3', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e16de068-2ab0-4de8-a06d-ce39ac551e5f-0', usage_metadata={'input_tokens': 13, 'output_tokens': 92, 'total_tokens': 105, 'input_token_details': {}, 'output_token_details': {}})

Once we have an llm client, we can wire togethe the retrieval component and the llm. First, let's try chaining the retriever to a prompt template, to get context aware prompts

In [11]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pprint import pprint

conversation_template = ChatPromptTemplate([
    ("system", """You will look at the following passages and reply to the question using only and only the information present in the following documents.
         If the information requested is not present in the documents or if the question can not be answered using the documents as context, just say that you can't answer the question
         ```{context}```
         """),
    ("human", """Question: ```{question}```
    Answer: """)
])

chain = {"context": retriever, "question": RunnablePassthrough()} | conversation_template

for message in chain.invoke("What is all I need?").messages:
  pprint((str(type(message)), message.content))

("<class 'langchain_core.messages.system.SystemMessage'>",
 'You will look at the following passages and reply to the question using only '
 'and only the information present in the following documents.\n'
 '         If the information requested is not present in the documents or if '
 'the question can not be answered using the documents as context, just say '
 "that you can't answer the question\n"
 "         ```[Document(metadata={'Header 2': '1 Introduction'}, "
 "page_content='## 1 Introduction'), Document(metadata={'Header 2': '5.2 "
 "Hardware and Schedule'}, page_content='## 5.2 Hardware and Schedule'), "
 "Document(metadata={'Header 2': '3 Model Architecture'}, page_content='## 3 "
 "Model Architecture'), Document(metadata={'Header 2': '3.1 Encoder and "
 "Decoder Stacks'}, page_content='## 3.1 Encoder and Decoder Stacks'), "
 "Document(metadata={'Header 2': '6.1 Machine Translation'}, page_content='## "
 "6.1 Machine Translation'), Document(metadata={'Header 2': '6.3 English 

Observe how the documents have a lot of irrelevant metadata.
We need to format them better using a helper function

In [12]:
from langchain_core.runnables import RunnableLambda

def format_message(msgs):
  return '\n\n'.join([f"Section: {next(iter(msg.metadata.values()))}\n Contents: {msg.page_content}" for msg in msgs])

chain = {"context": retriever | RunnableLambda(format_message), "question": RunnablePassthrough()} | conversation_template

for message in chain.invoke("What is all I need?").messages:
  pprint((str(type(message)), message.content))

("<class 'langchain_core.messages.system.SystemMessage'>",
 'You will look at the following passages and reply to the question using only '
 'and only the information present in the following documents.\n'
 '         If the information requested is not present in the documents or if '
 'the question can not be answered using the documents as context, just say '
 "that you can't answer the question\n"
 '         ```Section: 1 Introduction\n'
 ' Contents: ## 1 Introduction\n'
 '\n'
 'Section: 5.2 Hardware and Schedule\n'
 ' Contents: ## 5.2 Hardware and Schedule\n'
 '\n'
 'Section: 3 Model Architecture\n'
 ' Contents: ## 3 Model Architecture\n'
 '\n'
 'Section: 3.1 Encoder and Decoder Stacks\n'
 ' Contents: ## 3.1 Encoder and Decoder Stacks\n'
 '\n'
 'Section: 6.1 Machine Translation\n'
 ' Contents: ## 6.1 Machine Translation\n'
 '\n'
 'Section: 6.3 English Constituency Parsing\n'
 ' Contents: ## 6.3 English Constituency Parsing\n'
 '\n'
 'Section: Abstract\n'
 ' Contents: ## Abstract\n'

We can even implement filters such as filtering out the `References`

In [13]:
def format_message(msgs):
  return '\n\n'.join([f"Section: {next(iter(msg.metadata.values()))}\n Contents: {msg.page_content}" for msg in msgs if next(iter(msg.metadata.values())) != "References"])

chain = {"context": retriever | RunnableLambda(format_message), "question": RunnablePassthrough()} | conversation_template

for message in chain.invoke("What is this paper about?").messages:
  pprint((str(type(message)), message.content))

("<class 'langchain_core.messages.system.SystemMessage'>",
 'You will look at the following passages and reply to the question using only '
 'and only the information present in the following documents.\n'
 '         If the information requested is not present in the documents or if '
 'the question can not be answered using the documents as context, just say '
 "that you can't answer the question\n"
 '         ```Section: 1 Introduction\n'
 ' Contents: ## 1 Introduction\n'
 '\n'
 'Section: Attention Is All You Need\n'
 ' Contents: Llion Jones ∗ Google Research llion@google.com  \n'
 'Aidan N. Gomez ∗ † University of Toronto aidan@cs.toronto.edu  \n'
 'Łukasz Kaiser ∗ Google Brain lukaszkaiser@google.com  \n'
 'Illia Polosukhin ∗ ‡ illia.polosukhin@gmail.com\n'
 '\n'
 'Section: Provided proper attribution is provided, Google hereby grants '
 'permission to reproduce the tables and figures in this paper solely for use '
 'in journalistic or scholarly works.\n'
 ' Contents: ## Provided p

Now, present the conversation to the LLM and see what it responds

In [14]:
from langchain_core.output_parsers import StrOutputParser

chain = {"context": retriever | RunnableLambda(format_message), "question": RunnablePassthrough()} | conversation_template | llm | StrOutputParser()

print(chain.invoke("What is this paper about?"))

 This paper appears to be about the Transformer architecture, specifically in the context of machine translation. The title of the paper is "Attention Is All You Need." The authors listed in the paper include Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin, Ashish Vaswani, Noam Shazeer, Niki Parmar, and Jakob Uszkoreit. The paper includes a table (Table 3) comparing various variations on the Transformer architecture in terms of their performance on the English-to-German translation development set, newstest2013. The perplexities listed in the table are per-wordpiece.


We can customize the prompt to get additional details, such as quoting the source of the information from which it drew the conclusion from

In [15]:
conversation_template = ChatPromptTemplate([
    ("system", """You will look at the following passages and reply to the question using only and only the information present in the following documents.
         You will not refer to the information, just write the answer. After you give the answer, cite the section from which you read it.
         If the information requested is not present in the documents or if the question can not be answered using the documents as context, just say that you can't answer the question
         ```{context}```
         """),
    ("human", """Question: ```{question}```
    Answer: """)
])

chain = {"context": retriever | RunnableLambda(format_message), "question": RunnablePassthrough()} | conversation_template | llm | StrOutputParser()

print(chain.invoke("What is this paper about?"))

 This paper is about the Transformer model in the field of machine translation. [Section: Attention Is All You Need]


In [16]:
pprint(chain.invoke("Give me the summary of this work"))

(' The work described in the provided passages is titled "Attention Is All You '
 'Need" and was authored by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob '
 'Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin. '
 'The summary of this work is not explicitly stated, but it appears to be '
 'about a new attention function and its application in machine translation '
 'tasks. The attention function is described as a mapping of a query and a set '
 'of key-value pairs to an output, computed as a weighted sum. The work also '
 'discusses various model variations and the reasons for using self-attention '
 '(Section 4). However, the specifics of the self-attention mechanism, its '
 'advantages, and how it improves machine translation are not detailed in the '
 'provided passages. For a more comprehensive understanding, one would need to '
 'read the entire work or related explanatory materials. [Sections: 1 '
 'Introduction, 3.2 Attention, 4 Why Self-Attention, 6.

In [17]:
pprint(chain.invoke("What are the elements of the transformer architecture?"))

(' The Transformer architecture consists of stacked self-attention layers and '
 'point-wise, fully connected layers for both the encoder and decoder (Figure '
 '1, Section 3 Model Architecture). Additionally, the Transformer uses '
 'multi-head attention in three different ways (Section 3.2.3 Applications of '
 'Attention in our Model). Unfortunately, the documents do not provide more '
 'specific details about the number of layers, heads, or any other elements of '
 'the architecture.')


## Optional: Build a Gradio interface for our RAG pipeline

In [18]:
%load_ext gradio

In [19]:
%%blocks

import gradio as gr

# We need a function that calls our langchain pipeline

def rag(question):
  return chain.invoke(question)

demo = gr.Interface(
    fn=rag,             # Here, we are wiring the function to the interface
    inputs=["text"],      # Specify the input types
    outputs=["text"],     # Same, for output
)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

There you go.
Each RAG application has its own details and nuances. There is no good "one size fits all" solution. Instead, it depends heavily in my design choices.

This notebook provides a good starting point to implement your own RAG pipeline.